В машинном обучении есть два типа параметров:

* **Внутренние (параметры модели)** - Подбираются во время обучения и определяют, как использовать входные данные для получения необходимого результата. Например, это веса (коэффициенты уравнения) в линейной/логистической регрессии.

* **Внешние (параметры алгоритма)** - Их принято называть **гиперпараметрами**. Внешние параметры могут быть произвольно установлены перед началом обучения и контролируют внутреннюю работу обучающего алгоритма. Например, это параметр регуляризации в линейной/логистической регрессии. Гиперпараметры отвечают за сложность взаимосвязи между входными признаками и целевой переменной, поэтому сильно влияют на модель и качество прогнозирования.

Каждый алгоритм *МО* имеет набор гиперпараметров, которые определяют, как именно он строит модель на обучающей выборке. Например, в модуле *ML-2* для повышения эффективности модели мы уже рассматривали подбор параметра регуляризации $\alpha$ для алгоритма линейной регрессии *Ridge*.

In [1]:
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных
import matplotlib.pyplot as plt # для визуализации
import seaborn as sns # для визуализации
from sklearn import linear_model # линейные модели
from sklearn import metrics # метрики

```python
#Создаем список из 20 возможных значений от 0.001 до 10
alpha_list = np.linspace(0.01, 10, 20)
#Создаем пустые списки, в которые будем добавлять результаты 
train_scores = []
test_scores = []
for alpha in alpha_list:
    #Создаем объект класса линейная регрессия с L2-регуляризацией
    ridge_lr_poly = linear_model.Ridge(alpha=alpha, max_iter=10000)
    #Обучаем модель предсказывать логарифм целевого признака
    ridge_lr_poly.fit(X_train_scaled_poly, y_train_log)
    #Делаем предсказание для каждой из выборок
    #Если обучили на логарифме, то от результата необходимо взять обратную функцию - экспоненту
    y_train_predict_poly = np.exp(ridge_lr_poly.predict(X_train_scaled_poly))
    y_test_predict_poly = np.exp(ridge_lr_poly.predict(X_test_scaled_poly))
    #Рассчитываем метрику для двух выборок и добавляем их в списки
    train_scores.append(metrics.mean_absolute_error(y_train, y_train_predict_poly))
    test_scores.append(metrics.mean_absolute_error(y_test, y_test_predict_poly))
 
#Визуализируем изменение R^2 в зависимости от alpha
fig, ax = plt.subplots(figsize=(12, 4)) #фигура + координатная плоскость
ax.plot(alpha_list, train_scores, label='Train') #линейный график для тренировочной выборки
ax.plot(alpha_list, test_scores, label='Test') #линейный график для тестовой выборки
ax.set_xlabel('Alpha') #название оси абсцисс
ax.set_ylabel('MAE') #название оси ординат
ax.set_xticks(alpha_list) #метки по оси абцисс
ax.xaxis.set_tick_params(rotation=45) #поворот меток на оси абсцисс
ax.legend(); #отображение легенды
```

Результат:

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/47a7e0d5d24abca72171988571c18d13/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst-3-ml-7-2.png)

Наилучшее значение метрики соответствует $\alpha=0.01$ (кстати, можно попробовать перебрать значения $alpha < 0.01$).

В данном случае мы просто воспользовались циклом `for` и перебрали некоторые заданные значения *alpha*, хотя, по всей видимости, не самые оптимальные. Поэтому подобранные эмпирическим путём значения гиперпараметров с большей вероятностью дадут низкую прогностическую эффективность.

Также рассмотренный метод визуализации зависимости метрики от гиперпараметра позволяет выбрать только один внешний параметр, в данном случае — *alpha*. А что делать, если у нас не один, а несколько? 

Например, вспомним основные внешние параметры `DecisionTreeClassifier`:

* `criterion` — критерий информативности. Может быть равен `'gini'` — критерий Джини — и `'entropy'` — энтропия Шеннона.
* `max_depth` — максимальная глубина дерева. По умолчанию `None`, глубина дерева не ограничена.
* `max_features` — максимальное число признаков, по которым ищется лучшее разбиение в дереве. По умолчанию `None`, то есть обучение производится на всех признаках.
* `min_samples_leaf` — минимальное число объектов в листе. По умолчанию — 1.

Мы, конечно, можем сделать кучу вложенных циклов. Однако, поскольку поиск оптимальных значений гиперпараметров является общераспространенной задачей *МО*, библиотека *scikit-learn* и другие предлагают методы, позволяющие её решить.

> Тщательный подбор гиперпараметров гарантирует, что модель покажет максимально возможную точность на обучающих данных, но это **совершенно не означает хороший результат на тестовых или новых данных**.

# <center>Базовая оптимизация</center>

В базовой оптимизации, предоставляемой библиотекой *sklearn*, есть два основных метода — **grid search** и **random search**.

Наиболее часто используемый метод — это **поиск по сетке (grid search)**, который по сути является попыткой перебрать все возможные комбинации заданных гиперпараметров. Мы указываем список значений для различных гиперпараметров, и, ориентируясь на нашу метрику, оцениваем эффективность модели для каждого их сочетания, чтобы получить оптимальную комбинацию значений.

Допустим, мы хотим подобрать гиперпараметры `min_samples_leaf` и `max_depth` для алгоритма `DecisionTreeClassifier`. Зададим списки их значений:

```python
min_samples_leaf = [3, 5, 8, 9]
max_depth = [4, 5, 6, 7, 8]
```

Поскольку нам нужно перебрать четыре различных значения для `min_samples_leaf` и пять — для `max_depth`, то получается всего 4*5=20 комбинаций. Модель будет обучена 20 раз; столько же раз будет рассчитана метрика.

Мы перебираем множество значений гиперпараметров и выбираем ту комбинацию значений, которая даёт наилучшую точность на тестовых данных. Однако это **совсем не означает, что на новых данных мы получим такой же результат**. 

Поскольку мы использовали тестовый набор для настройки гиперпараметров, мы больше не можем использовать его для оценки качества модели. Теперь в этих целях нам необходим независимый набор данных, то есть набор, который не использовался для построения модели и настройки её гиперпараметров.

Следовательно, надо разбить данные на **три части**: **обучающую** для построения модели, **проверочную** (валидационную) для выбора гиперпараметров модели, а также **тестовую** для оценки качества модели и выбранных гиперпараметров. 

Для лучшей оценки обобщающей способности вместо одного разбиения данных на обучающий и проверочный наборы мы можем воспользоваться перекрёстной проверкой, то есть **кросс-валидацией (cross validation)**. В таком случае качество модели оценивается для каждой комбинации гиперпараметров по всем разбиениям кросс-валидации. 

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/761310cd7523c307072cadd085cf577e/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst-3-ml-7-4.png)

**Пояснение:**

Предположим, что у нас есть *n* комбинаций гиперпараметров. Берём первую комбинацию и обучаем на них первую модель с помощью кросс-валидации с 10 фолдами (*cv=10*), затем рассчитываем метрику как среднее по всем разбиениям. Так проделываем для каждой комбинации и выбираем ту, при которой наша метрика наилучшая. В итоге мы обучим *n*cv* моделей, но выберем один набор гиперпараметров, который и будет использоваться для обучения итоговой модели на всей обучающей выборке.

## <center>GridSearchCV</center>

Поскольку поиск по сетке с кросс-валидацией является весьма распространённым методом настройки гиперпараметров, библиотека *scikit-learn* предлагает класс `GridSearchCV`, в котором осуществляется именно такой вариант.

*P.S. Смотри ноутбук "extra: GridSearchCV"*

**Основные параметры `GridSearchCV`:**

* `estimator` — алгоритм, который будем оптимизировать;
* `param_grid` — словарь или список словарей. Словарь с именами гиперпараметров (в формате строки (*str*), например, `'max_depth'`) в качестве ключей и списками параметров (например, `[5, 8, 10]`) в качестве значений. Итого: `{'max_depth': [5, 8, 10] }`.
Также можно передать список таких словарей:
```python
param_grid = [
              {'max_depth': [5, 8, 10],
               'min_samples_leaf': [7, 8, 9] } #первый словарь 
               
              {'n_estimators': [100, 200, 300], 
               'max_depth': [5, 8, 10] } #второй словарь 
             ]
```
В таком случае каждый словарь в списке перебирается отдельно и последовательно.

* `scoring` — по умолчанию используется *score*-функция заданного алгоритма:
    * для классификации — `sklearn.metrics.accuracy_score`;
    * для регрессии — `sklearn.metrics.r2_score`;

* `cv` — количество фолдов в кросс-валидации, по умолчанию используется 5.
* `n_jobs` — количество ядер для распараллеливания расчёта. -1 использует все существующие ядра.

Чтобы воспользоваться классом `GridSearchCV`, необходимо:

1) Импортировать библиотеку:
```python
from sklearn.model_selection import GridSearchCV
```

2) Указать искомые гиперпараметры в виде словаря  `param_grid`: ключами словаря являются имена настраиваемых гиперпараметров, а значениями – тестируемые настройки гиперпараметров. Мы рассмотрим сетку из:
    * `'penalty'` — тип регуляризации. Может принимать значения `l1`,  `l2`, `'elasticnet'` или `None` (отсутствие регуляризации);
    * `'solver'` — алгоритм оптимизации, может принимать значения `'newton-cg'`, `'lbfgs'`, `'liblinear'`, `'sag'`, `'saga'`, по умолчанию — `'lbfgs'`.

> Важно помнить, что выбор алгоритма оптимизации зависит от выбранного типа штрафа:

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/843b8646e16b0a034ea479809c4a8d5c/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst-3-ml-7-5.png)

```python
param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
                  'solver': ['lbfgs', 'saga'] #алгоритм оптимизации
                  }
```

3) Вызвать класс `GridSearchCV` и передать модель (`LogisticRegression`), сетку искомых параметров (`param_grid`), а также число фолдов, которые мы хотим использовать в кросс-валидации, и `n_jobs = -1`, чтобы использовать все доступные ядра для расчётов:
```python
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=1, #генератор случайных чисел
        max_iter=1000 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)
```

4) Созданный нами объект `grid_search` аналогичен классификатору, поэтому мы можем вызвать стандартные методы `fit`, `predict` и `score` от его имени. Однако, когда мы вызываем `fit()`, он запускает кросс-валидацию для каждой комбинации гиперпараметров, указанных в `param_grid`:
```python
grid_search.fit(X_train_scaled, y_train) 
#Затраченное время: 1min 4s
```

> `GridSearchCV` включает в себя не только поиск лучших параметров, но и автоматическое построение новой модели на всём обучающем наборе данных с использованием параметров, которые дают наилучшее значение метрики при кросс-валидации.

Наилучшая найденная комбинация гиперпараметров сохраняется в атрибуте `best_params_`:
```python
print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
# Наилучшие значения гиперпараметров: {'penalty': 'none', 'solver': 'lbfgs'}
```

Наилучшая метрика:
```python
print("accuracy на тестовом наборе: {:.2f}".format(grid_search.score(X_test_scaled, y_test)))
```

Либо можем посмотреть любую другую метрику, воспользовавшись методом `predict()` и передав предсказанные значения в функцию для расчёта метрики (например, `f1_score()`):
```python
y_test_pred = grid_search.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
 
# accuracy на тестовом наборе: 0.84
# f1_score на тестовом наборе: 0.64
```

Значения метрик не изменились, но это значит лишь, что мы не нашли комбинацию внешних параметров лучше, чем заданы по умолчанию. Это неудивительно, и достаточно часто исходные  гиперпараметры дают неплохой результат, но это не повод останавливаться!

Попробуем расширить сетку гиперпараметров и проделаем те же шаги:

```python
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регуляризации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=1000), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
 
# %time - замеряет время выполнения
%time grid_search_1.fit(X_train_scaled, y_train) 
print("accuracy на тестовом наборе: {:.2f}".format(grid_search_1.score(X_test_scaled, y_test)))
y_test_pred = grid_search_1.predict(X_test_scaled)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))
 
#Затраченное время: 5min 43s (индивидуально, для каждого ПК)
#accuracy на тестовом наборе: 0.84
#f1_score на тестовом наборе: 0.64
#Наилучшие значения гиперпараметров: {'C': 0.3, 'penalty': 'l2', 'solver': 'lbfgs'}
```

Метрику опять не удалось улучшить, а времени потратили много, в пять раз больше!

Поиск по сетке не гарантирует, что мы найдём наилучшую комбинацию гиперпараметров. Всё потому, что сетка значений конечна, и фактическое наилучшее значение может отсутствовать или оказаться между значений, заданными нами.

Итоговая модель хранится в параметре `best_estimator_`, ей можно воспользоваться для получения прогнозов на новых данных: 
```python
print("Наилучшая модель:\n{}".format(grid_search.best_estimator_))
#Наилучшая модель: LogisticRegression(max_iter=1000, penalty='none', random_state=1)
```

А наилучшее значение метрики на кросс-валидации (значение метрики, усреднённое по всем разбиениям для данной комбинации гиперпараметров) хранится в атрибуте `best_score_`. 
```python
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_search.best_score_))
#Наилучшее значение точности при кросс-валидации: 0.84
```

> Не путайте `best_score_` со значением метрики модели, которое вычисляется на тестовом наборе с помощью метода `score`. Метод `score` (оценивающий качество результатов, полученных с помощью метода `predict()`) использует модель, построенную на всём обучающем наборе данных. В атрибуте `best_score_` записывается средняя метрика на кросс-валидации.

Результаты кросс-валидации хранятся в параметре `cv_results_`. Отрисуем, как менялась метрика при различных гиперпараметрах:
```python
visual = pd.pivot_table(pd.DataFrame(grid_search_1.cv_results_),
               values='mean_test_score', index='param_C',
               columns='param_solver')
sns.heatmap(visual)
plt.title('Тепловая карта зависимости метрики accuracy от solver и С') # подпись графика
sns.set(rc={'figure.figsize':(12, 8)}) #задаем размер графика
```

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/d2640828dc452cc42cf52eff9678b850/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst-3-ml-7-6.png)

Видим, что слабая регуляризация *С = 0.01* отрицательно влияет на метрику, поэтому есть смысл брать значения больше 0.5 и алгоритмы оптимизации *lbfgs* и *sag* работают лучше.

## <center>RandomizedSearchCV</center>

Альтернативным подходом подбора различных комбинаций гиперпараметров в библиотеке *scikit-learn* является `RandomizedSearchCV`. 

Рандомизированный поиск работает почти так же, как решётчатый поиск, за исключением того, что перебираются не всевозможные комбинации параметров, а из них случайным образом выбираются $n$ возможных вариантов комбинаций. Количество комбинаций параметров ($n$), которые используются в случайном поиске, мы задаём самостоятельно, что позволяет управлять временем, затрачиваемым на оптимизацию.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/7bc80431d813a9af9a6ec0409c53c4df/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst-3-ml-7-7.png)

* В `GridSearchCV` сетка задаётся вручную, перебираются различные значения гиперпараметров с каким-то шагом, в итоге получается что-то похожее на «красивую» сетку слева на картинке. Однако минимум функции (белое пятно) мы так и не обнаруживаем — а ведь он где-то рядом, возможно, просто между подобранными нами комбинациями.

* `RandomizedSearchCV` выбирает *n* (количество задаём сами) случайных точек/комбинаций из заданных нами последовательностей. Как следствие, мы можем перебирать не все возможные точки, а только часть из них, тем самым управляя скоростью работы перебора.

**Основные параметры `RandomizedSearchCV`** аналогичны `GridSearchCV`, за исключением наименований некоторых параметров и наличия параметра `n_iter`:

* `estimator` — алгоритм, который будем оптимизировать;
* `param_distributions` — cловарь с именами параметров (*str*) в качестве ключей и списками параметров в качестве значений, которые нужно попробовать.

> В ранних версиях *sklearn* данный параметр был обозначен как `param_grid` (как и в `GridSearchCV`).

Также можно передать список таких словарей:
```python
param_grid = [
              {'max_depth': [5, 8, 10],
               'min_samples_leaf': [7, 8, 9] } #первый словарь 
              {'n_estimators': [100, 200, 300], 
               'max_depth': [5, 8, 10] } #второй словарь 
             ]
```
В таком случае каждый словарь в списке перебирается отдельно и последовательно. Это позволяет выполнять поиск по любой последовательности настроек параметров.

* `scoring` — по умолчанию используется score-функция заданного алгоритма:
    * для классификации — `sklearn.metrics.accuracy_score`;
    * для регрессии — `sklearn.metrics.r2_score`.

* `cv` — количество фолдов в кросс-валидации, по умолчанию используется 5.
* `n_jobs` — количество ядер для распараллеливания расчёта. -1 использует все существующие ядра.
* `n_iter` — количество комбинаций на расчёт. От этого параметра напрямую зависит время оптимизации и качество модели.

*P.S. Смотри ноутбук "extra: RandomizedSearchCV"*

## <center>Рекомендации по настройке гиперпараметров ансамблей над решающими деревьями</center>

**Алгоритм случайного леса (RandomForest)**

* `n_estimators` — число итераций (количество деревьев). Частично работает правило «чем больше, тем лучше», но иногда это не имеет особого смысла и сильно увеличивает затраты, поэтому стоит пробовать обучать сотни деревьев [100,200, 300, 400]. Если нет изменений, то оставить минимальное — 100.

* `max_depth` — максимальная глубина дерева. В случайном лесе строятся «сильные» деревья, каждое из которых даёт полноценный прогноз, поэтому глубина деревьем может быть достаточно большой. Стоит следить за переобучением.

* `max_features` — максимальное количество признаков, учитываемых алгоритмом при поиске лучшего разделения;

* `max_samples` — доля выборки, которая будет использоваться для обучения каждого алгоритма — дерева.

**Алгоритм градиентного бустинга (GradientBoosting)**

* `n_estimators` — число итераций (количество деревьев) : хотя ошибка на обучении монотонно стремится к нулю, ошибка на контроле, как правило, начинает увеличиваться после определенной итерации. Оптимальное число итераций можно выбирать, например, по отложенной выборке или с помощью кросс-валидации.

* `learning_rate` — темп обучения (0;1]. Как правило, чем меньше темп обучения, тем лучше качество итоговой композиции.

* `max_depth` — максимальная глубина дерева. Используется для борьбы с переобучением. Рекомендуется устанавливать не более 5.

* `max_features` — максимальное количество признаков, учитываемых алгоритмом при поиске лучшего разделения.

* `subsample` — доля выборки, которая будет использоваться для обучения каждого алгоритма. Это ещё один способ улучшения качества градиентного бустинга. Таким образом вносится рандомизация в процесс обучения базовых алгоритмов, что снижает уровень шума в обучении, а также повышает эффективность вычислений. 

> **Рекомендация**. Берите подвыборки, размер которых вдвое меньше исходной выборки.

> Главное отличие техник *Bagging* и *Boosting* состоит в параллельном и последовательном построении деревьев соответственно.

![image.png](https://lms-cdn.skillfactory.ru/assets/courseware/v1/45f50e8761202094e73aa7c1fa399014/asset-v1:Skillfactory+DSMED+2023+type@asset+block/dst-3-ml-7-8.png)

> Общепринятая практика для бустинга — подгонять `n_estimators` в зависимости от бюджета времени и памяти, а затем подбирать различные значения `learning_rate`.

# <center>Продвинутая оптимизация</center>